# Qwen3 0.8B Training Template - Colab

**Purpose:** Proven template for training Qwen3 on A100 GPU

**This template works!** Created after successful Phase 1 (Linux Commands) training.

---

## 📝 How to Use This Template:

1. **Copy this file** and rename for your phase (e.g., `phase2_python_training.ipynb`)
2. **Update configuration cell** (Cell 4):
   - `MODEL_NAME` - base model or merged model path
   - `OUTPUT_DIR` - unique output directory name
   - `DATASET_NAME` - your HuggingFace dataset
3. **Customize dataset formatting** (Cell 9) if needed
4. **Run all cells** - training will start automatically

---

## ⚙️ Proven Configuration:
- **GPU:** A100 (40GB)
- **Training Time:** ~4.5 hours for 3 epochs
- **Batch Size:** 4 (effective 16 with gradient accumulation)
- **LoRA:** r=16, alpha=32
- **Memory:** 4-bit quantization + gradient checkpointing
- **Padding Token:** 151645 (CRITICAL - don't change!)

---

## 🎯 Tested Phases:
- ✅ **Phase 1:** Linux Commands (AnishJoshi/nl2bash-custom) - SUCCESS!
- 📝 **Phase 2:** Python System Automation - Ready to test
- 📝 **Phase 3:** Advanced Python - Ready to test

In [ ]:
# Install dependencies
!pip install -q transformers datasets accelerate peft bitsandbytes trl torch

print("\n✅ Installation complete")

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# ⚙️ Configuration

**EDIT THIS CELL for each new phase:**
- Change `MODEL_NAME` if using merged model from previous phase
- Change `OUTPUT_DIR` to unique name (e.g., `qwen3-phase2-python`)
- Change `DATASET_NAME` to your HuggingFace dataset
- Adjust `NUM_EPOCHS`, `BATCH_SIZE` if needed (current settings work great!)

**DO NOT CHANGE:**
- `PAD_TOKEN_ID` - MUST be 151645 for consistency
- LoRA config values (proven to work)

In [ ]:
# Configuration - EDIT THIS FOR EACH PHASE
MODEL_NAME = "DavidAU/Qwen3-Zero-Coder-Reasoning-0.8B"  # Change to merged model path if sequential
OUTPUT_DIR = "./qwen3-phase1-output"  # Change for each phase
DATASET_NAME = "AnishJoshi/nl2bash-custom"  # Change to your dataset

# Training hyperparameters (proven to work on A100)
BATCH_SIZE = 4
GRADIENT_ACCUMULATION = 4
LEARNING_RATE = 2e-4
NUM_EPOCHS = 3
MAX_SEQ_LENGTH = 2048

# LoRA config (don't change - this works!)
LORA_R = 16
LORA_ALPHA = 32
LORA_DROPOUT = 0.05

# CRITICAL: Same padding token across ALL phases
PAD_TOKEN_ID = 151645

print("✅ Configuration set")
print(f"   Model: {MODEL_NAME}")
print(f"   Output: {OUTPUT_DIR}")
print(f"   Dataset: {DATASET_NAME}")

In [ ]:
# Load tokenizer
print("Loading tokenizer from HuggingFace...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

# CRITICAL: Force same padding token
tokenizer.pad_token_id = PAD_TOKEN_ID
tokenizer.padding_side = "right"

print(f"✅ Tokenizer loaded")
print(f"   Pad token ID: {tokenizer.pad_token_id}")

In [ ]:
# Load model with 4-bit quantization
print("="*60)
print("Loading Model from HuggingFace")
print("="*60)

print("\nConfiguring 4-bit quantization...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

print(f"Loading {MODEL_NAME}...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

print("Preparing model for LoRA training...")
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

print(f"✅ Model loaded on device: {model.device}")

In [ ]:
# Configure LoRA
print("\nApplying LoRA configuration...")
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
print("\n✅ LoRA applied")
model.print_trainable_parameters()

In [ ]:
# Load dataset
print("Loading dataset...")
dataset = load_dataset(DATASET_NAME, split="train")
print(f"Dataset size: {len(dataset)} examples")
print(f"\nSample entry:")
print(dataset[0])

# 📝 Dataset Formatting

**CUSTOMIZE THIS CELL for your dataset structure:**

Different datasets have different field names:
- Some use `description` + `cmd`
- Some use `question` + `answer`
- Some use `instruction` + `output`

Update the `format_instruction()` function to match your dataset's structure.

In [ ]:
# Format dataset - CUSTOMIZE THIS FOR YOUR DATASET STRUCTURE
def format_instruction(example):
    # Adjust field names based on your dataset
    description = example.get('description', example.get('question', example.get('instruction', '')))
    command = example.get('cmd', example.get('answer', example.get('output', '')))
    
    prompt = f"Instruction: {description}\n\nResponse:"
    text = f"{prompt} {command}"
    return {"text": text}

dataset = dataset.map(format_instruction)
print("\n✅ Dataset formatted")
print("\nExample:")
print(dataset[0]['text'][:500])

In [ ]:
# Training arguments
training_args = SFTConfig(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    learning_rate=LEARNING_RATE,
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
    optim="paged_adamw_8bit",
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    report_to="none",
    dataset_text_field="text",
    packing=False,
)

print("✅ Training arguments configured")

In [ ]:
# Create trainer and start training
print("\n" + "="*60)
print("STARTING TRAINING")
print("="*60)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

print(f"\nTraining on {len(dataset)} examples...")
print(f"Total steps: {len(dataset) // (BATCH_SIZE * GRADIENT_ACCUMULATION) * NUM_EPOCHS}")
print("\nStarting training...\n")

trainer.train()

print("\n" + "="*60)
print("✅ TRAINING COMPLETE!")
print("="*60)

In [ ]:
# Ensure model is on GPU for saving (fixes potential issues)
model = model.to("cuda")
print(f"✅ Model moved to GPU: {next(model.parameters()).device}")

In [ ]:
# Save the trained LoRA adapter
print("\nSaving LoRA adapter...")
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

import os
adapter_path = f"{OUTPUT_DIR}/adapter_model.safetensors"
if os.path.exists(adapter_path):
    adapter_size = os.path.getsize(adapter_path) / 1e6
    print(f"✅ Adapter saved: {adapter_size:.1f} MB")
    print(f"   Location: {OUTPUT_DIR}")
else:
    print(f"❌ ERROR: Adapter file not found at {adapter_path}")
    
# List all saved files
print("\n📁 Saved files:")
for file in os.listdir(OUTPUT_DIR):
    size = os.path.getsize(f"{OUTPUT_DIR}/{file}") / 1e6
    print(f"   {file}: {size:.2f} MB")

# ✅ Training Complete!

## Next Steps:

1. **Download the adapter** from Colab Files panel:
   - Navigate to your `OUTPUT_DIR` folder
   - Download `adapter_model.safetensors` (should be ~40-50MB)
   
2. **Save this notebook** to Google Drive (File → Save)

3. **Sync to local machine:**
   ```bash
   cp -rfv /home/archlinux/GoogleDrive/* /home/archlinux/code_insiders/ml_ai_engineering/colab_drive/
   ```

4. **For next phase:**
   - Copy this template
   - Update configuration cell
   - Run all cells

---

## 📊 Performance Notes:
- Record your training time
- Note any errors or issues
- Compare with Kaggle T4 performance